# Live Trading

In [1]:
import alpyen.brokerinterface as alp_bi
import alpyen.livetrading as alp_lt
import alpyen.signal as alp_sig
import alpyen.strategy as alp_str
import alpyen.utils as alp_utils
import alpyen.backtesting as alp_bt
import alpyen.datacontainer as alp_dc

In [2]:
from typing import List, Dict
from eventkit import Event

# Signal and Strategy for debugging live trading
class TempSignal(alp_sig.SignalBase):
    _signal_signature = 'TS'

    def __init__(self,
                 signature_str: str,
                 input_data_array: List[Event],
                 warmup_length: int) -> None:
        self._signal_name = input_data_array[0].name() + "_TS_" + str(warmup_length)
        self._signal_event = Event(self._signal_name)

    def calculate_signal(self) -> int:
        prices = self.get_data_by_name(self._input_data_array[0].name())
        return round(prices[-1]/0.00001, 0) % 10
    
class TempStrategy(alp_str.StrategyBase):
    _strategy_signature = 'TempStrategy'

    def __init__(self,
                 signature_str: str,
                 input_signal_array: List[Event],
                 trade_combo: alp_str.TradeCombos,
                 warmup_length: int,
                 initial_capital: float = 100.0,
                 order_manager: alp_bi.OrderManagerBase = None) -> None:
        self._strategy_name = 'My temp strategy'


    def make_order_decision(self) -> Dict[str, float]:
        signal_name = next(iter(self._signal_storage))
        if self._signal_storage[signal_name][-1] >= 0 and self._signal_storage[signal_name][-1] < 2:
            return {'combo_1': 2.0}
        elif self._signal_storage[signal_name][-1] >= 2 and self._signal_storage[signal_name][-1] < 4:
            return {'combo_1': -2.0}
        elif self._signal_storage[signal_name][-1] >= 4 and self._signal_storage[signal_name][-1] < 6:
            return {'combo_2': 3.0}
        elif self._signal_storage[signal_name][-1] >= 6 and self._signal_storage[signal_name][-1] < 8:
            return {'combo_2': -3.0}
        else:
            return {'combo_3': 1.0}

In [3]:
input_tickers = ['EURUSD']
trade_tickers = ['EURUSD', 'GBPUSD', 'CHFUSD']
signal_info_dict = {}

for ticker in input_tickers:
    signal_info_dict[ticker + '_TS_1'] = alp_utils.SignalInfo('TS', [ticker],
                                                          [alp_utils.ContractType.FX],
                                                          [alp_utils.PriceOHLCType.Close], 2, {})

# Subscribe to strategies
strategy_info_dict = {}
strategy_name = 'Temp_Strategy'
trade_combo = {'combo_1': [-50.0, 100.0, 0.0],
               'combo_2': [-100.0, 0.0, -60.0],
               'combo_3': [0.0, -100.0, 70.0]}
strategy_info_dict[strategy_name] = alp_utils.StrategyInfo(
    'TempStrategy', [ticker + '_TS_1' for ticker in input_tickers],
    1, {},
    trade_tickers, [alp_utils.ContractType.FX] * 3, trade_combo)

In [10]:
import threading
import asyncio

ib_api = alp_bi.BrokerAPIBase('IB')
ib_api.connect(port=4002)
    
# Create live trader
class JupyterLiveTrader(alp_lt.LiveTrader):
    def start_trading(self) -> None:
        """
        Start live trading.
        """
        with self.setup() as (signal_dict, strategy_dict):
            event_loop = asyncio.new_event_loop()

            task = event_loop.create_task(self.async_sleep())
            event_loop.run_until_complete(task)
    
# Create live trader
path_to_control_file = 'test_control_nb.yml'
my_trader = JupyterLiveTrader('IB', signal_info_dict, strategy_info_dict, path_to_control_file)
my_trader.start_trading()


Peer closed connection. clientId 1 already in use?
API connection failed: TimeoutError()


TimeoutError: 

Error 1300, reqId -1: Socket port has been reset and this connection is being dropped. Please reconnect on the new port -4003
Peer closed connection.


In [11]:
my_trader.get_is_trading()

True

In [12]:
portfolio_df = my_trader.get_portfolio_manager().get_portfolio_info()
portfolio_df

,strategy_name,combo_name,combo_position,combo_entry_price,combo_mtm_price,unrealized_pnl,realized_pnl


In [8]:
my_trader.disconnect()

# Backtesting

In [ ]:
# Read data
data_folder_path = 'tests\\Data\\'
ticker_name = 'BBH'
short_lookback = 5
long_lookback = 200
short_lookback_name = ticker_name + '_MA_' + str(short_lookback)
long_lookback_name = ticker_name + '_MA_' + str(long_lookback)
ticker_names = [ticker_name]
all_input = alp_dc.DataUtils.aggregate_yahoo_data(ticker_names, data_folder_path)

In [ ]:
# Subscribe to signals
signal_info_dict = {}
signal_info_dict[short_lookback_name]\
    = alp_utils.SignalInfo('MA', ticker_names, [], [], short_lookback, {})
signal_info_dict[long_lookback_name]\
    = alp_utils.SignalInfo('MA', ticker_names, [], [], long_lookback, {})

In [ ]:
# Subscribe to strategies
strategy_info_dict = {}
strategy_name = ticker_name + '_MACrossing_01'
strategy_info_dict[strategy_name] = alp_utils.StrategyInfo(
    'MACrossing',
    [short_lookback_name, long_lookback_name],
    1, {}, ticker_names, combo_definition={'combo1': [1.0]})

In [ ]:
# Create backtester
number_path = 100
my_backtester = alp_bt.Backtester(all_input, ticker_names, signal_info_dict, strategy_info_dict,
                                       number_path)
my_backtester.run_backtest()
backtest_results = my_backtester.get_results()

In [ ]:
backtest_results = my_backtester.get_results()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Distribution of Sharpe ratio
plt.hist(backtest_results[strategy_name][str(alp_bt.MetricType.PoorMansSharpeRatio)])

# Portfolio

In [2]:
ib_api = alp_bi.BrokerAPIBase('IB')
ib_api.connect(port=4002)

In [3]:
account_value_list = ib_api.get_handle().accountSummary()
print(account_value_list)

ib_api.get_handle().sleep(3)

[AccountValue(account='DU1786171', tag='AccountType', value='INDIVIDUAL', currency='', modelCode=''), AccountValue(account='DU1786171', tag='Cushion', value='0.972743', currency='', modelCode=''), AccountValue(account='DU1786171', tag='LookAheadNextChange', value='1642429800', currency='', modelCode=''), AccountValue(account='DU1786171', tag='AccruedCash', value='-687.29', currency='USD', modelCode=''), AccountValue(account='DU1786171', tag='AvailableFunds', value='954852.42', currency='USD', modelCode=''), AccountValue(account='DU1786171', tag='BuyingPower', value='6365682.81', currency='USD', modelCode=''), AccountValue(account='DU1786171', tag='EquityWithLoanValue', value='997813.37', currency='USD', modelCode=''), AccountValue(account='DU1786171', tag='ExcessLiquidity', value='970646.13', currency='USD', modelCode=''), AccountValue(account='DU1786171', tag='FullAvailableFunds', value='954852.42', currency='USD', modelCode=''), AccountValue(account='DU1786171', tag='FullExcessLiquid

True

In [4]:
my_portfolio = ib_api.get_handle().portfolio()
print(my_portfolio)

ib_api.get_handle().sleep(3)

[PortfolioItem(contract=Forex('CHFUSD', conId=12087802, right='0', primaryExchange='IDEALPRO', localSymbol='CHF.USD', tradingClass='CHF.USD'), position=467680.0, marketPrice=1.09592495, marketValue=512542.19, averageCost=1.094, unrealizedPNL=900.27, realizedPNL=0.0, account='DU1786171'), PortfolioItem(contract=Forex('EURUSD', conId=12087792, right='0', primaryExchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), position=680590.0, marketPrice=1.14312505, marketValue=777999.48, averageCost=1.13457335, unrealizedPNL=5820.22, realizedPNL=0.0, account='DU1786171'), PortfolioItem(contract=Forex('GBPUSD', conId=12087797, right='0', primaryExchange='IDEALPRO', localSymbol='GBP.USD', tradingClass='GBP.USD'), position=347900.0, marketPrice=1.368425, marketValue=476075.06, averageCost=1.35622305, unrealizedPNL=4245.06, realizedPNL=0.0, account='DU1786171'), PortfolioItem(contract=Stock(conId=756733, symbol='SPY', right='0', primaryExchange='ARCA', currency='USD', localSymbol='SPY'

True

In [5]:
my_positions = ib_api.get_handle().positions()
print(my_positions)

ib_api.get_handle().sleep(3)

[Position(account='DU1786171', contract=Forex('GBPUSD', conId=12087797, localSymbol='GBP.USD', tradingClass='GBP.USD'), position=347900.0, avgCost=1.35622305), Position(account='DU1786171', contract=Forex('EURUSD', conId=12087792, localSymbol='EUR.USD', tradingClass='EUR.USD'), position=680590.0, avgCost=1.13457335), Position(account='DU1786171', contract=Stock(conId=756733, symbol='SPY', exchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), position=-100.0, avgCost=321.513226), Position(account='DU1786171', contract=Forex('CHFUSD', conId=12087802, localSymbol='CHF.USD', tradingClass='CHF.USD'), position=467680.0, avgCost=1.094)]


True

In [6]:
ib_api.disconnect()

AccountValue(account='DU1786171', tag='LookAheadNextChange', value='1642429800', currency='', modelCode='')